<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
# Import SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Carla_Ferreira_API_challenge').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

In [3]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [7]:
#Schemas
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)
                             ])


lines_schema = StructType([StructField('_corrupt_record', StringType(), True),
                           StructField('color', StringType(), True),
                           StructField('facilities', StringType(), True),
                           StructField('id', StringType(), True),
                           StructField('localities', StringType(), True),
                           StructField('long_name', StringType(), True),
                           StructField('municipalities', StringType(), True),
                           StructField('patterns', StringType(), True),
                           StructField('routes', StringType(), True),
                           StructField('short_name', StringType(), True),
                           StructField('text_color', StringType(), True)
                           ])

municipalities_schema = StructType([StructField('district_id', StringType(), True),
                                    StructField('district_name', StringType(), True),
                                    StructField('id', StringType(), True),
                                    StructField('name', StringType(), True),
                                    StructField('prefix', StringType(), True),
                                    StructField('region_id', StringType(), True),
                                    StructField('region_name', StringType(), True)
                                    ])

In [8]:
vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles", schema = vehicle_schema)
lines = readFromAPI("https://api.carrismetropolitana.pt/lines", schema = lines_schema)
municipalities = readFromAPI("https://api.carrismetropolitana.pt/municipalities", schema = municipalities_schema)

In [27]:
# create "date" extracted from "timestamp" column (format: hh24miss) -- this format doesnt make sense, the correct is yyyyMMdd
from pyspark.sql.functions import date_format
vehicles = vehicles.withColumn("date", date_format('timestamp', "yyyyMMdd"))

vehicles.show()

+--------------------+-------+----------+----+--------------------+--------------------+---------------+--------------------+--------------------+----------+----------+
|     _corrupt_record|  color|facilities|  id|          localities|           long_name| municipalities|            patterns|              routes|short_name|text_color|
+--------------------+-------+----------+----+--------------------+--------------------+---------------+--------------------+--------------------+----------+----------+
|                NULL|#C61D23|        []|1001|["Alfragide","Ama...|Alfragide (Estr S...|       ["1115"]|["1001_0_1","1001...|          ["1001_0"]|      1001|   #FFFFFF|
|                NULL|#C61D23|        []|1002|["Reboleira","Ama...|Reboleira (Estaçã...|       ["1115"]|        ["1002_0_3"]|          ["1002_0"]|      1002|   #FFFFFF|
|                NULL|#C61D23|        []|1003|["Amadora","Amado...|Amadora (Estação ...|       ["1115"]|["1003_0_1","1003...|          ["1003_0"]|      100

In [26]:
from pyspark.sql import functions as F

vehicles.select("date").distinct().agg(F.min("date").alias("min_date")).show()
vehicles.select("date").distinct().agg(F.max("date").alias("max_date")).show()

# there's only one day in this file.

+--------+
|min_date|
+--------+
|20241121|
+--------+

+--------+
|max_date|
+--------+
|20241121|
+--------+



Write data as PARQUET into the BRONZE layer (/content/lake/bronze)

*   Partition "vehicles" by "date" column
*   Paths: /content/lake/bronze/vehicles, /content/lake/bronze/lines
, /content/lake/bronze/municipalities

Make sure there is only 1 single parquet created /
Use overwrite as write mode

Declare the directory to write the data that came from API carrismetropolitana

In [20]:
!mkdir -p /content/lake/bronze

path_vehicles = "/content/lake/bronze/vehicles"
path_lines = "/content/lake/bronze/lines"
path_municipalities = "/content/lake/bronze/municipalities"

In [21]:
(vehicles
.coalesce(1)
.write
.mode("overwrite")
.partitionBy("date")
.format("parquet")
.save(path_vehicles)
)


(lines
.write
.mode("overwrite")
.format("parquet")
.save(path_lines)
)


(municipalities
.write
.mode("overwrite")
.format("parquet")
.save(path_municipalities)
)